Introduction

The dataset we will be working along with is Seattle Public Library Inventory dataset. We have downloaded the dataset from the source and perfomed cleaning operation before uploading into the data. Post that we use pymysql to connect to the database and perform operation like recommending books to user based on the genre and find the book in the inventory based on the authors name.


In [ ]:
#Importing the essential libraries

import pandas as pd (#pandas dataframe)
import pymysql (#python package to connect to aws instance)
import re (#regular expression to filter our columns)
import matplotlib.pyplot as plt,%matplotlib inline (#python visualization libraries)


In [ ]:
#Using pandas to read the csv file of library inventory
df=pd.read_csv('Library_Collection_Inventory.csv')

c1=pd.read_csv('../input/seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_2005.csv')
c2=pd.read_csv('../input/seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_2006.csv')
c3=pd.read_csv('../input/seattle-library-checkout-records/Checkouts_By_Title_Data_Lens_2007.csv')

#cleaning the checkout column to parse it as dates

c1.CheckoutDateTime.replace(to_replace=r'PM', value=' ', regex=True,inplace=True)
ab=c1['CheckoutDateTime'].str.split(" ",n=1,expand=True)
c1['Date']=ab[0]
c1['Date']=pd.to_datetime(c1['Date'])

c2.CheckoutDateTime.replace(to_replace=r'PM', value=' ', regex=True,inplace=True)
ab=c2['CheckoutDateTime'].str.split(" ",n=1,expand=True)
c2['Date']=ab[0]
c2['Date']=pd.to_datetime(c2['Date'])
c2.drop('CheckoutDateTime',axis=1,inplace=True)

c3.CheckoutDateTime.replace(to_replace=r'PM', value=' ', regex=True,inplace=True)
ab=c3['CheckoutDateTime'].str.split(" ",n=1,expand=True)
c3['Date']=ab[0]
c3['Date']=pd.to_datetime(c3['Date'])
c3.drop('CheckoutDateTime',axis=1,inplace=True)

In [ ]:
#Merging all the files
f=pd.concat([c1,c2])
f=pd.concat([f,c3])

In [ ]:
#Performing join operations on f and library inventory to get if book was in print or media format

new=pd.DataFrame(df[['Title','Author','Subjects','ItemType']])
temp=pd.DataFrame(f['BibNumber'])
dd=pd.read_csv('../input/seattle-library-checkout-records/Integrated_Library_System__ILS__Data_Dictionary.csv')

new=pd.merge(left=f,right=pd.DataFrame(dd[['Code','Format Group']]),left_on='ItemType',right_on='Code')

In [ ]:
#Performing join operations on new and library inventory to get author, tile, subjects and Itemtype details

a = df.drop_duplicates(subset='BibNum', keep='last').reset_index()

new=pd.merge(left=new,right=pd.DataFrame(a[['BibNum','Title','Author','Subjects']]),left_on='BibNumber',right_on='BibNum')


In [ ]:
#Removing Unwanted columns

new.drop(['BibNum','Collection','CallNumber','ItemBarcode'],axis=1,inplace=True)
new.drop(['Code'],axis=1,inplace=True)

In [ ]:
#Using date column to find new paramters like year month and dayofweek 

new['Year']=new['Date'].dt.year
new['Month']=new['Date'].dt.month
new['Week']=new['Date'].dt.weekofyear
new['DayofWeek']=new['Date'].dt.dayofweek

#Removing date column as we have segreated its information into new columns

new.drop('Date',axis=1,inplace=True)

In [ ]:
#Now we have to divide our dataset into different categories, but the subject column does not give us proper value
#We will separate this by identifying top occuring genres using regular expression and use them
#below is the code that plot the top occuring genres in subjects columns


plt.figure(figsize=(20,8)) #plotting

#plot chart
ax1 = plt.subplot(121, aspect='equal')

#save top 10 occuring genres in 'a'
a=pd.DataFrame(pd.Series(' '.join(map(str,df['Subjects'])).lower().split()).value_counts()[:10],columns=['total'])

#plot 'a' as pie chart to show the % distribution

a.plot(kind='pie', y = 'total', ax=ax1, autopct='%1.1f%%', 
startangle=90, shadow=False, labels=a.index, legend = False, fontsize=14)

In [ ]:
#We will use the 4 most occuring genres and
#Add them in the Subject columns

#Adding the columns and intializing them as 0
cols=['fiction','mystery','drama','literature']
for col in cols: a[col] = 0

In [ ]:
#We will use below script to find if the new genres columns created exist in each row and if they do
#make its value 1

#Since we have 5lakh rows we will perform operation in parts
#Using iterrows to get each row in pandas dataframe


for index,row in pd.DataFrame(a[(a.index>50000) & (a.index<=100000)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1


for index,row in pd.DataFrame(a[(a.index>100000) & (a.index<=200000)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1


for index,row in pd.DataFrame(a[(a.index>200000) & (a.index<=300000)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1
            

for index,row in pd.DataFrame(a[(a.index>300000) & (a.index<=400000)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1
            

for index,row in pd.DataFrame(a[(a.index>400000) & (a.index<=500000)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1
            

for index,row in pd.DataFrame(a[(a.index>500000) & (a.index<=584391)].Subjects).iterrows():
   
    for col in cols:
        if re.search(col,str(row['Subjects']).lower()):
            
            a.ix[index, col] = 1            

In [ ]:
#Now we will map the new columns in our checkout dataset

new=pd.merge(left=new,right=pd.DataFrame(a[['BibNum','fiction','mystery','drama','literature']]),
             left_on='BibNumber',right_on='BibNum')

new.to_csv('Checkout.csv')
a.to_csv('Inventory.csv')

In [ ]:
#We will no upload our clean data into the databse

#BEFORE that below is the conf file used to setup nodes and establish connections between them

[mysqld]
datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock
bind-address=0.0.0.0
user=mysql

default_storage_engine=InnoDB
innodb_autoinc_lock_mode=2
innodb_flush_log_at_trx_commit=0
innodb_buffer_pool_size=128M

binlog_format=ROW
log-error=/var/log/mysqld.log

[galera]
wsrep_on=ON
wsrep_provider=/usr/lib/galera/libgalera_smm.so

wsrep_node_name='galera3'

wsrep_node_address="172.31.41.172"
wsrep_cluster_name='galera-training'
wsrep_cluster_address=gcomm://172.31.32.84,172.31.42.23,172.31.41.172

wsrep_provider_options="gcache.size=300M; gcache.page_size=300M"
wsrep_slave_threads=4
wsrep_sst_method=rsync



#We have already trasnfered our cleaned csv file to aws instance using WINSCP

#Next we will create a database and a table in SQL to import this data

create database project;

create table checkout(
BibNumber int(20), 
ItemType varchar(10),
Format Group varchar(10),
Title varchar(150),
Author varchar(150), 
Subjects varchar(150),
Year int(10), 
Month int(10), 
Week int(10), 
DayofWeek int(5),  
fiction int(5), 
mystery int(5),
drama, int(5) 
literature int(5)

);

create table inventory(
BibNum int(20), 
Title varchar(150),
Author varchar(150),
Publisher varchar(150),
Subjects varchar(150),
ItemType varchar(10),

);

#Now we will load the csv file into this new table created in our database

load data local infile '/home/ubuntu/Checkout.csv' into table checkout fields terminated by ',' Enclosed by '"' Lines terminated by '\n' Ignore 1 rows;

load data local infile '/home/ubuntu/Inventory.csv' 
into table inventory fields terminated by ',' Enclosed by '"' Lines terminated by '\n' Ignore 1 rows;

In [ ]:
#Now we will establish a connection with database and write an api which will perform two functions
#1 it will recommend book to users based on their genre preference
#2 it will help them find book by author even if they do not specify full name

#Establishing connection with the database
host="18.223.3.204" #host ip adress
port=3306 #port on which we will establish a connection
dbname="project" #name of database
user="root" #username for accessing db
password="host" #password for accessing db

conn = pymysql.connect(host, user=user,port=port,
                           passwd=password, db=dbname) #create a connection between python api and aws instance

In [ ]:
#Importing the dataset
new=pd.DataFrame(pd.read_sql('select * from checkout', con=conn))
a=pd.DataFrame(pd.read_sql('select * from inventory', con=conn))

In [ ]:
genre=raw_input("Enter the year and genre you want to have statistics for: ")

def statistics(year, genre):

#We will find the statistics of which book was most checked out in each genre
#Performing groupby operation to remove duplicate values
    s=new[(new['Year']==year)&& (new[genre]==genre)].groupby('BibNumber').sum().sort_values('count',ascending=False)
#removing duplicate value
    s.drop_duplicates(inplace=True)
#merging to get information of Author and book tile
    s=pd.merge(left=s,right=pd.DataFrame(new[['BibNumber','Author','Title']]),left_on='BibNumber',right_on='BibNumber')
#display entire row
    pd.set_option('display.max_colwidth', -1)
#Ourput the result
    Print('The most checked out book in fiction was: ')
    return pd.DataFrame(s[['Title','Author']].head(1))

#We will find the statistics like which book was least checked out in each genre
#Performing groupby operation to remove duplicate values
    s=new[(new['Year']==year)&& (new[genre]==genre)].groupby('BibNumber').sum().sort_values('count',ascending=True)
#removing duplicate value
    s.drop_duplicates(inplace=True)
#merging to get information of Author and book tile
    s=pd.merge(left=s,right=pd.DataFrame(new[['BibNumber','Author','Title']]),left_on='BibNumber',right_on='BibNumber')
#display entire row
    pd.set_option('display.max_colwidth', -1)
#Ourput the result
    Print('The least checked out book in fiction was: ')
    return pd.DataFrame(s[['Title','Author']].head(1))


In [ ]:
#Recommending books to users based on checkout records

#Taking input for genre

genre=raw_input("Enter the genre you want to have suggestions for: ")

def suggest(genre):

#We will find the book which were checkedout most in users mentioned genre and recommend top 10 books to user
    s=new[new[genre]==1].groupby('BibNumber').sum().sort_values('count',ascending=False)
    s=pd.merge(left=s,right=pd.DataFrame(new[['BibNumber','Author','Title']]),left_on='BibNumber',right_on='BibNumber')
    s.drop_duplicates(inplace=True)
    pd.set_option('display.max_colwidth', -1)
    return pd.DataFrame(s[['Title','Author']].head(10))

In [ ]:
#Finding books for user based on author name even if partial

#Taking input for author

author=raw_input("Enter the author you want to search for: ")

#Function to find books based on author
def find(author):
    #Using iterrows to traverese each row in datafram
    for index,row in a.iterrows():
        #Condition to check if author name is matching with user keywords
        if re.search(str(author),str(row['Author'])):
            pd.set_option('display.max_colwidth', -1) #display entire cell
            print(pd.DataFrame(row[['Title','Author','Publisher']])) #Print title,author and publish for matching authors